In [3]:
import tensorflow as tf
import pandas as pd
from math import sqrt
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

In [8]:
df_classification = pd.read_csv('D:/ML/data/classification/airlines_task_preprocessed.csv', index_col=0)
df_regression = pd.read_csv('D:/ML/data/regression/moldova_cars_task_preprocessed.csv', index_col=0)
y_cf = df_classification[["Delay"]]
X_cf = df_classification.drop(["Delay"], axis=1)

mms = MinMaxScaler()
mms.fit(X_cf)
X_cf = mms.transform(X_cf)

y_rs = df_regression[["Price(euro)"]]
X_rs = df_regression.drop(["Price(euro)"], axis=1)

mms = MinMaxScaler()
mms.fit(X_rs)
X_rs = mms.transform(X_rs)

X_train_cf, X_test_cf, y_train_cf, y_test_cf = train_test_split(X_cf, y_cf, test_size=0.2, random_state=4)
X_train_rs, X_test_rs, y_train_rs, y_test_rs = train_test_split(X_rs, y_rs, test_size=0.2, random_state=4)

w0 = 1 / y_train_cf[y_train_cf==0].shape[0]
w1 = 1 / y_train_cf[y_train_cf==1].shape[0]

In [25]:
model_regression = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(4, activation="relu", input_shape=(X_train_rs.shape[1],)),
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dense(12, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dense(4, activation="relu"),
        tf.keras.layers.Dense(4, activation="relu"),
        tf.keras.layers.Dense(2, activation="linear"),
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mae")
model_regression.fit(X_train_rs, y_train_rs, epochs=10)

Epoch 1/10
925/925 [==============================] - 1s 787us/step - loss: 5419.4302
Epoch 2/10
925/925 [==============================] - 1s 730us/step - loss: 4691.5361
Epoch 3/10
925/925 [==============================] - 1s 737us/step - loss: 4482.9570
Epoch 4/10
925/925 [==============================] - 1s 838us/step - loss: 3854.2559
Epoch 5/10
925/925 [==============================] - 1s 731us/step - loss: 3434.6067
Epoch 6/10
925/925 [==============================] - 1s 734us/step - loss: 3366.1519
Epoch 7/10
925/925 [==============================] - 1s 734us/step - loss: 3316.4141
Epoch 8/10
925/925 [==============================] - 1s 729us/step - loss: 3278.9807
Epoch 9/10
925/925 [==============================] - 1s 771us/step - loss: 3257.4824
Epoch 10/10
925/925 [==============================] - 1s 722us/step - loss: 3252.2083


In [20]:
y_pred = model_regression.predict(X_test_rs)
print(f'MAE: {mean_absolute_error(y_test_rs, y_pred)}')
print(f'MSE: {mean_squared_error(y_test_rs, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test_rs, y_pred))}')
print(f'MAPE: {(mean_absolute_percentage_error(y_test_rs, y_pred))}')
print(f'R^2: {r2_score(y_test_rs, y_pred)}')

232/232 [==============================] - 0s 523us/step
MAE: 2836.202201538829
MSE: 71084783.6641899
RMSE: 8431.179257031006
MAPE: 0.39319112878680645
R^2: 0.4896871340873018


In [11]:
model_classification = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(X_train_cf.shape[1],)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)
model_classification.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="sparse_categorical_crossentropy")
model_classification.fit(X_train_cf, y_train_cf, epochs=5,
                           class_weight={0: w0, 1: w1})

Epoch 1/5
3612/3612 [==============================] - 4s 882us/step - loss: 5.4522e-06
Epoch 2/5
3612/3612 [==============================] - 3s 875us/step - loss: 5.3467e-06
Epoch 3/5
3612/3612 [==============================] - 3s 878us/step - loss: 5.3119e-06
Epoch 4/5
3612/3612 [==============================] - 3s 877us/step - loss: 5.2931e-06
Epoch 5/5
3612/3612 [==============================] - 3s 877us/step - loss: 5.2806e-06


In [12]:
predictions = [np.argmax(pred) for pred in model_classification.predict(X_test_cf, verbose=None)]
print(accuracy_score(y_test_cf, predictions))
print(confusion_matrix(y_test_cf, predictions))
print(classification_report(y_test_cf, predictions))

0.6762415642844782
[[15656  2430]
 [ 6925  3884]]
              precision    recall  f1-score   support

           0       0.69      0.87      0.77     18086
           1       0.62      0.36      0.45     10809

    accuracy                           0.68     28895
   macro avg       0.65      0.61      0.61     28895
weighted avg       0.66      0.68      0.65     28895

